Baixar repositório *script* de avaliação

In [27]:
!git clone https://bitbucket.org/HBrum/tweetsentbr.git
%cd tweetsentbr/sent-analysis/
!pip install nlpnet

Cloning into 'tweetsentbr'...
remote: Counting objects: 85, done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 85 (delta 20), reused 0 (delta 0)
Unpacking objects: 100% (85/85), done.
/content/tweetsentbr/sent-analysis/mt-dnn_port/Training/tweetsent/tweetsentbr/sent-analysis


Extrair arquivos de avaliação

In [34]:
!unzip tweetSentBR_extracted.zip

Archive:  tweetSentBR_extracted.zip
   creating: tweetSentBR_extracted/
  inflating: tweetSentBR_extracted/testTT.neg  
  inflating: tweetSentBR_extracted/testTT.neu  
  inflating: tweetSentBR_extracted/testTT.pos  
  inflating: tweetSentBR_extracted/trainTT.neg  
  inflating: tweetSentBR_extracted/trainTT.neu  
  inflating: tweetSentBR_extracted/trainTT.pos  
  inflating: tweetSentBR_extracted/tweets.none  
  inflating: tweetSentBR_extracted/tweets.neg  
  inflating: tweetSentBR_extracted/tweets.neu  
  inflating: tweetSentBR_extracted/tweets.pos  


Obter arquivo com resultados

In [0]:
filepath = '../../results/results.csv'

Gerar arquivo único de teste separado por tabulação

In [24]:
from io import StringIO
import pandas as pd
import os
import re

corpora = [f for f in os.listdir('tweetSentBR_extracted') if 'testTT' in f]

print('Separando as colunas por tabulação...')
tabbed = dict()
for goldpath in corpora:
    with open("tweetSentBR_extracted/{}".format(goldpath)) as f:
        text = f.read()

    assert '\t' not in text 
    
    outtext = re.sub(r'(.+?) (.+)',r'\1\t\2', text)
    tabbed.update({goldpath:StringIO(outtext)})
    print(goldpath)
    
print('Unificando os arquivos..')

header = ['id', 'premise']
abbr = {'neg': 'Negative', 'neu': 'Neutral', 'pos': 'Positive'} 
output = pd.DataFrame()

for path, f in tabbed.items():
    table = pd.read_csv(f, sep = '\t', names = header)

    posfix = path.split('.')[1]
    label = abbr[posfix]
    table['label'] = label

    output = table if output.empty else output.append(table)

output.to_csv('tweetsentbr_test.csv', index = None)
print('tweetsentbr_test.csv')

Separando as colunas por tabulação...
testTT.neu
testTT.neg
testTT.pos
Unificando os arquivos..
tweetsentbr_test.csv


Gerar *scores*

In [32]:
from classify import report
import pandas as pd

corpus  = pd.read_csv('tweetsentbr_test.csv')
result = pd.read_csv(filepath)

distances = result['distances']
number = {'Negative': 0, 'Neutral': 1, 'Positive': 2}
predictions = result['predictions'].map(number)

y_test = list()
for result_idx, result_row in result.iterrows():
    uid = result_row['TweetID']
    corpus_row  = corpus.loc[corpus['id'] == uid]
    y = corpus_row.iloc[0]['label']
    y_test.append(number[y])

report(distances, predictions, y_test)

Acc: 0.7119
F1. neg: 0.703 neu: 0.585 post: 0.792
Acc dist min: 0.0000
Acc dist max: 0.0000
Acc dist min equal: 0.0000
Acc dist max equal: 0.0000


/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:61: FutureWarning: 
The current behaviour of 'Series.argmin' is deprecated, use 'idxmin'
instead.
The behavior of 'argmin' will be corrected to return the positional
minimum in the future. For now, use 'series.values.argmin' or
'np.argmin(np.array(values))' to get the position of the minimum
row.
  return bound(*args, **kwds)
/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:61: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  return bound(*args, **kwds)
